Rough Draft for the take home final. 

In [1]:
!pip install xmltodict


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY", "COP", "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW", "KWD", "LYD", "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN", "QAR", "RUB", "SAR", "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]
ratesForBase = [r for r in rates if r != "USD" and r != "EUR" and r != "GBP"]

Code Given by Professor Powers in the Prompt. 

In [3]:
import requests
import xmltodict
import json
import random

# URL of the XML data
date = "2011-05-04"
base = random.choice(ratesForBase)
url = f"https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date={date}&base_currency_code={base}&format_type=xml"
print(url)
# Fetch the XML data
response = requests.get(url)
response.raise_for_status()  # Ensure we notice bad responses

# Parse the XML data to a Python dictionary
data_dict = xmltodict.parse(response.text)

# Convert the dictionary to a JSON string
json_data = json.dumps(data_dict, indent=4)

# Print the JSON data
print(json_data)

# Optionally, write the JSON data to a file
with open(f"{date}_exchange_rates_{base}.json", "w") as json_file:
    json_file.write(json_data)

https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date=2011-05-04&base_currency_code=BND&format_type=xml
{
    "channel": {
        "title": "XML Historical Foreign Exchange Rates for Brunei Dollar (BND) (4 May 2011)",
        "link": "http://www.floatrates.com/currency/bnd/",
        "xmlLink": "http://www.floatrates.com/daily/bnd.xml",
        "description": "XML historical foreign exchange rates for Brunei Dollar (BND) from the Float Rates. Published at 4 May 2011.",
        "language": "en",
        "baseCurrency": "BND",
        "pubDate": "Wed, 4 May 2011",
        "lastBuildDate": "Wed, 4 May 2011",
        "item": [
            {
                "title": "1 BND = 0.546214 EUR",
                "link": "http://www.floatrates.com/bnd/eur/",
                "description": "1 Brunei Dollar = 0.546214 Euro",
                "pubDate": null,
                "baseCurrency": "BND",
                "baseName": "Brunei Dollar",
